In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
%%time
adatas_pk = []
for ti in [6,8,10,12,14,17,21]:
    print(ti)
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk.obs = adata_pk.obs.set_index('cell')
    print(adata_pk.var['peak'][:3].values)
    
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

qs_avc = []
l2fc_avc = []
for cond_order, obssub in adata.obs.groupby('cond_order'):
    # get sub
    adatasub = adata[obssub.index]
    adata_pk = adatas_pk[cond_order]
    
    # get A vs C 
    x = adatasub.obsm['pca_p17on'][...,0]
    cond_a = x < np.percentile(x,20)
    cond_c = x > np.percentile(x,80)
    cells_a = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_a].obs.index.values])
    cells_c = np.array([c.split(' ')[0][:-len('-2023')] for c in adatasub[cond_c].obs.index.values])
    
    cells_atac = adata_pk.obs.index.values
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    mat_a = np.log10(1+adata_pk_a.X[...]) # layers['lognorm'][...]
    mat_c = np.log10(1+adata_pk_c.X[...]) # adatasub_c.layers['lognorm'][...]
    
    # DEGs
    lfc = np.log2(10)*(np.mean(mat_c, axis=0) - np.mean(mat_a, axis=0)) # log2FC (log10CPM as raw counts)
    ts, ps = stats.ttest_ind(mat_a, mat_c)
    rs, qs, _, _ = multipletests(np.nan_to_num(ps, nan=1).reshape(-1,), method='fdr_bh') # why nan in ps -- not expressed
    
    num_sig = np.sum(np.logical_and(qs < 0.05, np.abs(lfc) > 1))
    print(cond_order, mat_a.shape, mat_c.shape, num_sig)
    
    # save this
    qs_avc.append(qs)
    l2fc_avc.append(lfc)
    
    # break
    
qs_avc = np.array(qs_avc)
l2fc_avc = np.array(l2fc_avc)

In [ ]:
# adata.uns['qs_avc'] = qs_avc
# adata.uns['l2fc_avc'] = l2fc_avc
# adata

In [ ]:
l2fc_th = 1
alpha_th = 0.05
cond_sig   = np.logical_and(qs_avc < alpha_th, np.abs(l2fc_avc) > l2fc_th)
cond_sig_c = np.logical_and(qs_avc < alpha_th, l2fc_avc >  l2fc_th)
cond_sig_a = np.logical_and(qs_avc < alpha_th, l2fc_avc < -l2fc_th)

print('num AvsC-DEGs per cond:\t', cond_sig.sum(axis=1))
print('num A-DEGs per cond:\t',    cond_sig_a.sum(axis=1))
print('num C-DEGs per cond:\t',    cond_sig_c.sum(axis=1))

In [ ]:
gene_sig_instances   = cond_sig.sum(axis=0)
gene_sig_instances_a = cond_sig_a.sum(axis=0)
gene_sig_instances_c = cond_sig_c.sum(axis=0)

instances, counts = np.unique(gene_sig_instances, return_counts=True)
instances_a, counts_a = np.unique(gene_sig_instances_a, return_counts=True)
instances_c, counts_c = np.unique(gene_sig_instances_c, return_counts=True)

print('num AvsC-DEGs in num conds:\t',  counts  , np.any(cond_sig, axis=0).sum())
print('num A-DEGs in num conds:\t',     counts_a, np.any(cond_sig_a, axis=0).sum())
print('num C-DEGs in num conds:\t',     counts_c, np.any(cond_sig_c, axis=0).sum())

In [ ]:
%%time
fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_avc_p6to21.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_avc_p6to21.txt'

np.savetxt(fout1, qs_avc)
np.savetxt(fout2, l2fc_avc)

In [ ]:
a = np.loadtxt(fout1)
a.shape

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.plot(np.arange(1,1+7), counts_a[1:], '-o', label='A', color='C0')
ax.plot(np.arange(1,1+7), counts_c[1:], '-o', label='C', color='C2')
ax.set_ylim(ymin=0)
ax.legend()
ax.set_ylabel('number of TFs')
ax.set_xlabel('number of time points')
sns.despine(ax=ax)
plt.show()

In [ ]:
# next steps
# volcanos
# nearest genes
# tf enrichments